In [2]:
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df
import numpy as np

## Loading the whole raw dataset.

In [3]:
# all_performances = pd.read_csv("perf-history/ny_phil.csv")

## Converting string dates to datetimes; creating seperate date related columns; converting programID to integer.

In [ ]:
all_performances['Date'] =  pd.to_datetime(head['Date'])

all_performances['year'] = all_performances['Date'].dt.year
all_performances['month'] = all_performances['Date'].dt.month
all_performances['day'] = all_performances['Date'].dt.day

# Dropping the initial column as no longer relevant.
all_performances.drop("Date", inplace=True, axis=1)

all_performances = all_performances.astype({"programID": int})

## Cleaning the data from meaningless rows such as rows not having to a programID, and not having any soloist. 
### This removed ~ 40k rows.

In [ ]:
all_performances = all_performances[all_performances.programID.notnull()]
all_performances = all_performances[all_performances.soloistName.notnull() 
                                    & all_performances.soloistRoles.notnull() & all_performances.soloistInstrument.notnull()]

## Creating utility functions to re-format date and time, location column and add season data.

In [ ]:
def split_location(location):
    
    split_ = location.split(",", 1)
    city = "-"
    state = "-"
    country = "-"
    try:
        if len(split_[1].strip()) > 2:
            country = split_[1].strip().title()
            city = split_[0].strip()
        else:
            city = split_[0].strip()
            state = split_[1].strip()
            country = "USA"
        return [country, city, state]
    except:
        city = location.title()
        return [country, city, state]

In [ ]:
def split_starting_time(time):
    
    split_ = time.split(":")
    hour = None
    minute = None
    
    if time == "None" or not time:
        return ["None", "None"]
    
    try:
        # Checking if last two elements of time 
        # is AM and first two elements are 12 
        if time[-2:] == "AM" and time[:2] == "12": 
            hour = "00"
            minute = time.split(":")[1].strip("AM")
#             return "00" + time[2:-2] 

        # remove the AM     
        elif time[-2:] == "AM": 
            hour = time.split(":")[0]
            minute = time.split(":")[1].strip("AM")

        # Checking if last two elements of time 
        # is PM and first two elements are 12    
        elif time[-2:] == "PM" and time[:2] == "12": 
            hour = time.split(":")[0]
            minute = time.split(":")[1].strip("PM")

        else: 
            hour = str(int(time[:1]) + 12)
            minute = time[2:].strip("PM").strip("AM")
            
        return [hour, minute]

    except:
        return time

In [ ]:
def seasonOfYear(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Autumn"
    else:
        return "Undefined"

## Applying the functions defined above to the dataset.

In [ ]:
all_performances['Country'] = all_performances['Location'].apply(lambda x: split_location(x)[0])
all_performances['City'] = all_performances['Location'].apply(lambda x: split_location(x)[1])
all_performances['State'] = all_performances['Location'].apply(lambda x: split_location(x)[2])

all_performances['startingHour'] = all_performances['Time'].apply(lambda x: split_starting_time(x)[0])
all_performances['startingMinute'] = all_performances['Time'].apply(lambda x: split_starting_time(x)[1])

all_performances['seasonOfYear'] = all_performances['month'].apply(lambda x: seasonOfYear(x))

# Drop the now useless columns.
all_performances.drop("Location", inplace=True, axis=1) 
all_performances.drop("soloistRoles", inplace=True, axis=1) 
all_performances.drop("Time", inplace=True, axis=1) 

## Creating geo-code mapping for the graph vizualisation.

In [ ]:
def make_geocode(country):
    
    if country in {"USA"}:
        return "USA"
    elif country in {"Canada"}:
        return "CAN"
    elif country == "France":
        return "FRA"
    elif country == "Netherlands":
        return "NLD"
    elif country in {"England", "Scotland"}:
        return "GBR"
    elif country == "Hungary":
        return "HUN"
    elif country == 'Czech Republic':
        return "CZE"
    elif country in {'Germany'}:
        return "DEU"
    elif country == 'Switzerland':
        return "CHE"
    elif country == 'Panama':
        return "PAN"
    elif country == 'Venezuela':
        return "VEN"
    elif country == 'Colombia':
        return "COL"
    elif country == 'Ecuador':
        return "ECU"
    elif country == 'Peru':
        return "PER"
    elif country == 'Paraguay':
        return "URY"
    elif country == 'Chile':
        return "CHL"
    elif country == 'Uruguay':
        return "URY"
    elif country == 'Argentina':
        return "ARG"
    elif country == 'Greece':
        return "GRC"
    elif country == 'Turkey':
        return "TUR"
    elif country == 'Austria':
        return "AUT"
    elif country == 'Poland':
        return "POL"
    elif country in {'Russia'}:
        return "RUS"
    elif country == 'Serbia':
        return "SRB"
    elif country == 'Croatia':
        return "HRV"
    elif country in {'Italy'}:
        return "ITA"
    elif country == 'Norway':
        return "NOR"
    elif country == 'Finland':
        return "FIN"
    elif country == 'Sweden':
        return "SWE"
    elif country == 'Japan':
        return "JPN"
    elif country == 'Belgium':
        return "BEL"
    elif country == 'Israel':
        return "ISR"
    elif country == 'Portugal':
        return "PRT"
    elif country in {'Spain'}:
        return "ESP"
    elif country == 'New Zealand':
        return "NZL"
    elif country == 'Australia':
        return "AUS"
    elif country == 'Denmark':
        return "DNK"
    elif country in {'South Korea'}:
        return "KOR"
    elif country == 'Dominican Republic':
        return "DOM"
    elif country == 'Mexico':
        return "MEX"
    elif country == 'Brazil':
        return "BRA"
    elif country == 'Taiwan':
        return "TWN"
    elif country == 'China':
        return "CHN"
    elif country == 'Thailand':
        return "THA"
    elif country == 'Luxembourg':
        return "LUX"
    elif country == 'The Philippines':
        return "PHL"
    elif country == 'Malaysia':
        return "MYS"
    elif country == 'Puerto Rico':
        return "PRI"
    elif country == 'Vietnam':
        return "VNM"
    elif country == 'United Arab Emirates':
        return "ARE"
    elif country in {"India"}:
        return "IND"
    elif country == "Indonesia":
        return "IDN"
    elif country == "Ireland":
        return "IRL"
    else:
        return "-"
    
geo_codes = {country: make_geocode(country) for country in nyphil.Country.unique()}  
country_per_code = {v: k for k, v in geo_codes.items()}

## Creating separate location dataframes, one per state and per country and applying the geo-code mapping.

In [ ]:
location_events = nyphil[nyphil['geocode'] != "-"].groupby('geocode')['programID'].nunique().reset_index(name="count").sort_values(
    by="count", ascending=True)
location_events_state = nyphil[nyphil['State'] != "-"].groupby('State')['programID'].nunique().reset_index(name="count").sort_values(
    by="count", ascending=True)
location_events_per_country["country"] = location_events_per_country['geocode'].apply(lambda x: country_per_code[x])

In [ ]:
all_yr_dfs_state = []
for y in nyphil.year.unique():
    testyrdf = nyphil[(nyphil['State'] != "-") & (nyphil['year'] == y)].groupby(['State', 'year'])['programID'].nunique().reset_index(name="count").sort_values(by="count", ascending=True)
    all_yr_dfs_state.append(testyrdf)

big_location_aggregation_state = pd.concat(all_yr_dfs_state)

## Scaling the counts to logarithm of it, to facilitate better visuzaliation due to outliers.

In [ ]:
big_location_aggregation_state['log_count'] = big_location_aggregation_state['count'].apply(lambda x: np.log(x) if np.log(x) != 0 else 0.5)
big_location_aggregation['log_count'] = big_location_aggregation['count'].apply(lambda x: np.log(x) if np.log(x) != 0 else 0.5)